# GraphRAG Relevance, Factuality and Synthesis Evaluation

## Step 0: environment set up

In [11]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index
import pandas as pd
from conn import connect2Googlesheet,retrieval_rel_docs, get_concatenate_df, apply_metric
from libs import context_builder, chunk_finder, enhanced_chunk_finder
# Force reload of the .env file
load_dotenv()

True

In [12]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD")
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")

Connected to Neo4j database successfully.


## Step 1: Create vector index

In [13]:
#create_vector_index(graph, "entities")

## Step 2: Load questions from google sheet

In [14]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
worksheet = spreadsheet.get_worksheet(2)  

# Get all records as a list of dictionaries
data = worksheet.get_all_records()

# Convert to Pandas DataFrame
df_MedQ = pd.DataFrame(data)
df_MedQ.head()

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blocking agents increases the ventilator free days?,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated with neuromuscular blocking agents have increased muscle weakness?,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?","Maybe this question: In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?",fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have a significance difference in mortality rate beween patients who recieved an early and continous cisatracurium infusion than those with usual care approach with lighter sedation targets?,Local question (not sure if this is the aim of your project) It will be nice as second step after proving the general summarization is working but focusing in general summarization would be priority in my opinion so you can have meanigful tool.,fixed,Wrong concept since PEEP by itself is mandatory component in ventilator.,"Does the use of neuromuscular blockers in patients with moderate-to-severe ARDS impact cardiovascular stability, particularly in terms of vasopressor requirements and hemodynamic effects, compared to sedation strategy without routine neuromuscular blockade?"
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservative fluid management strategy improves lung function and decrease ventilator days compared to liberal strategy?","Local question (not sure if this is the aim of your project) consider (WikiJournal): In patients with ALI/ARDS that are intubated and receiving positive pressure ventilation, how does the conservative compare to the liberal fluid management strategy in reducing mortality?",fixed,Check if studies defined conservative by CVP < 4 or elese just dont mention how much the CVP (i prefer the last approach),


## Step 3: Relevance check for top K questions

In [ ]:
def retrieval_rel_docs (graph, questions, top_k=5):
    top_k_questions = questions.head(top_k)
    # Initialize a list to store the results
    results = []
    # Iterate over the top k questions
    for index, row in top_k_questions.iterrows():
        question_number = index + 1  # Assuming the question number is the index + 1
        question = row['Question']  # Replace 'Question' with the actual column name for questions in df_MedQ
        
        # Generate response for the question
        # context = context_builder(graph, question, method="vector")
        filenames , output = enhanced_chunk_finder(graph, question,similarity_threshold = 0.9)
        # Extract relevant documents from the response content
        # docs = response.choices[0].message.content  # Adjust this based on the actual response structure
        # Iterate over the output to extract chunk details
        for chunk in output:
            file_name, chunk_text, page_number, position , similarity = chunk
            # Append the result to the list
            results.append({
                'Question number': question_number,
                'Question': question,
                'Retrieved FileName': file_name,
                'Chunk Text': chunk_text,
                'Page Number': page_number,
                'Position': position,
                'Similarity': similarity
            })
        # Append the result to the list
        #results.append({'Question number': question_number, 'Question': question, 'Retrieved FileName': filenames, 'Chunks': output})

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results, columns=[
        'Question number', 'Question', 'Retrieved FileName', 'Chunk Text', 'Page Number', 'Position' , 'Similarity'
    ])
    
    return results_df , filenames
#retrieval_rel_docs (graph, questions, top_k=1)

In [19]:
# Set pandas display options to show the full text content
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
topk = 5
results_df, filenames= retrieval_rel_docs(graph, df_MedQ, top_k=topk)
results_df
# results_df.to_csv('./outputs/retrieved_docs_results.csv', index=False)

,Question number,Question,Retrieved FileName,Chunk Text,Page Number,Position,Similarity
0,2,Do patients with severe ARDS being treated with neuromuscular blocking agents have increased muscle weakness?,ACURASYS.pdf,"Steroidal compounds (vecuronium, pancuronium, and ro­ cu­ronium) may carry the highest risk of myopa- thy,23 although myopathy has also been reported with benzylisoquinolines, including cisatracuri- um besylate.24,25 Muscle weakness was not in- creased significantly by the use of the neuro- muscular blocking agent in our study. The short duration of use of the neuromuscular blocking agent probably explains this result. In conclusion, this multicenter trial provides evidence that the administration of a neuromuscu- lar blocking agent early in the course of severe ARDS managed with low-tidal-volume ventilation may improve outcomes. Future studies are need- ed to replicate and expand these findings before they can be widely adopted in clinical practice. Supported by the Assistance Publique–Hôpitaux de Marseille and a grant from the Ministère de",9,59,0.930769
1,2,Do patients with severe ARDS being treated with neuromuscular blocking agents have increased muscle weakness?,ACURASYS.pdf,"care unit patients who require neuromuscular-blocking agents. Crit Care Med 1995;23:450-8. Testelmans D, Maes K, Wouters P, et 23.\t al. Rocuronium exacerbates mechanical ventilation-induced diaphragm dysfunc- tion in rats. Crit Care Med 2006;34:3018- 23. Davis NA, Rodgers JE, Gonzalez ER, 24.\t Fowler AA III. Prolonged weakness after cisatracurium infusion: a case report. Crit Care Med 1998;26:1290-2. Leatherman JW, Fluegel WL, David 25.\t WS, Davies SF, Iber C. Muscle weakness in mechanically ventilated patients with severe asthma. Am J Respir Crit Care Med 1996;153:1686-90. Copyright © 2010 Massachusetts Medical Society. posting presentations at medical meetings on the internet Posting",10,73,0.930769
2,2,Do patients with severe ARDS being treated with neuromuscular blocking agents have increased muscle weakness?,ACURASYS.pdf,"Neuromuscular Blocking Agents in ARDS n engl j med 363;12 nejm.org september 16, 2010 1115 The mechanisms underlying the beneficial effect of neuromuscular blocking agents remain speculative. A brief period of paralysis early in the course of ARDS may facilitate lung-protective mechanical ventilation by improving patient– ventilator synchrony and allowing for the accurate adjustment of tidal volume and pressure levels, thereby limiting the risk of both asynchrony- related alveolar collapse and regional alveolar- pressure increases with overdistention. Another possible mechanism of the benefit involves a de- crease in lung or systemic inflammation.15 The main safety concern with the use of a neuromuscular blocking agent is muscle weak- ness; the risk varies among agents.21,22 Steroidal compounds (vecuronium, pancuronium, and ro­ cu­ron",9,58,0.930769
